# 🧪 Análisis Exploratorio de Datos (EDA)
## Desempleo Región de Los Ríos (SU2) - Encuesta Nacional de Empleo (ENE)

### Introducción

Este notebook realiza un análisis exploratorio de los indicadores de desocupación en la Región de Los Ríos, segmentando por sexo y tramo etario, utilizando datos de la Encuesta Nacional de Empleo (ENE) del INE.  
El objetivo es identificar patrones, tendencias y grupos de mayor vulnerabilidad, para fundamentar análisis posteriores y recomendaciones de política pública.

---

### Carga y Revisión de Datos

- **Fuente:** INE - Encuesta Nacional de Empleo (ENE)
- **Cobertura:** Región de Los Ríos, 2010-2025
- **Variables clave:** periodo, valor, dimensión (sexo/tramo etario), categoría

Se verifica la estructura, duplicados y valores nulos para asegurar la calidad del dataset antes

In [50]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Configurar paleta de colores DALTÓNICA: AMARILLO, AZUL y GRIS
# Colores específicamente optimizados para daltonismo con alta visibilidad
economist_colors = [
    '#FFD700',  # Amarillo dorado - máxima visibilidad para daltonismo
    '#1E88E5',  # Azul brillante - alto contraste con amarillo
    '#757575',  # Gris medio - neutral y distinguible
    '#FFC107',  # Amarillo ámbar - variación del amarillo
    '#2196F3',  # Azul material - variación del azul
    '#9E9E9E',  # Gris claro - variación del gris
    '#FFEB3B',  # Amarillo limón - tercera variación
    '#0D47A1',  # Azul oscuro - cuarta variación
    '#616161',  # Gris oscuro - cuarta variación
    '#F57F17'   # Amarillo oscuro - quinta variación
]

# Validar que tenemos suficientes colores
print(f"✅ Paleta daltónica AMARILLO-AZUL-GRIS cargada: {len(economist_colors)} colores")
print("🎨 Colores principales: AMARILLO (#FFD700), AZUL (#1E88E5), GRIS (#757575)")
print("📊 Optimizada para protanomalía, deuteranomalía y tritanomalía")

# Configurar el template personalizado estilo The Economist
def apply_economist_style(fig):
    """Aplica el estilo The Economist a una figura de Plotly con tipografía Georgia"""
    fig.update_layout(
        font=dict(family='Georgia, serif', color='#1e293b'),
        title=dict(
            font=dict(size=18, family='Georgia, serif', color='#1e293b'),
            x=0.02,  # Alinear a la izquierda como The Economist
            xanchor='left'
        ),
        xaxis=dict(
            title=dict(font=dict(family='Georgia, serif')),
            tickfont=dict(family='Georgia, serif'),
            showgrid=True,
            gridcolor='#E5E5E5',
            gridwidth=0.5
        ),
        yaxis=dict(
            title=dict(font=dict(family='Georgia, serif')),
            tickfont=dict(family='Georgia, serif'),
            showgrid=True,
            gridcolor='#E5E5E5',
            gridwidth=0.5
        ),
        legend=dict(
            font=dict(family='Georgia, serif'),
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=60, r=40, t=80, b=60)
    )
    return fig

# Cargar datos
df = pd.read_csv("../data/processed/poblacion_desocupada_combinado.csv")
print(f"📊 Dataset cargado: {len(df)} registros")
df.head()

✅ Paleta daltónica AMARILLO-AZUL-GRIS cargada: 10 colores
🎨 Colores principales: AMARILLO (#FFD700), AZUL (#1E88E5), GRIS (#757575)
📊 Optimizada para protanomalía, deuteranomalía y tritanomalía
📊 Dataset cargado: 15292 registros


,indicador,periodo,valor,region,dimension,categoria
0,DES,2010-01-01/P3M,420.763708,11,tramo_edad,Y35T44
1,DES,2010-02-01/P3M,244.385161,11,tramo_edad,Y35T44
2,DES,2010-03-01/P3M,531.478140,11,tramo_edad,Y35T44
3,DES,2010-04-01/P3M,650.276185,11,tramo_edad,Y35T44
4,DES,2010-05-01/P3M,789.708634,11,tramo_edad,Y35T44


### 📋 Revisión de Calidad y Estructura de los Datos

A continuación, se revisa la estructura y calidad del dataset utilizando los siguientes comandos:


In [2]:
df.info()
df.duplicated().sum()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15292 entries, 0 to 15291
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   indicador  15292 non-null  object 
 1   periodo    15292 non-null  object 
 2   valor      15292 non-null  float64
 3   region     15292 non-null  object 
 4   dimension  15292 non-null  object 
 5   categoria  15292 non-null  object 
dtypes: float64(1), object(5)
memory usage: 716.9+ KB


indicador    0
periodo      0
valor        0
region       0
dimension    0
categoria    0
dtype: int64

## 🕵️‍♂️ Distribución Temporal del Desempleo

Visualizamos la evolución del desempleo desde 2010 hasta 2025.

In [51]:
# Preparar datos temporales
df_copy = df.copy()
df_copy["periodo"] = pd.to_datetime(df_copy["periodo"].str[:10])

# Calcular total de desocupados por periodo
df_temporal = df_copy.groupby("periodo")["valor"].sum().reset_index()

# Crear gráfico con elementos visuales mejorados
fig = go.Figure()

# Línea principal con relleno
fig.add_trace(go.Scatter(
    x=df_temporal["periodo"],
    y=df_temporal["valor"],
    mode='lines+markers',
    line=dict(color=economist_colors[0], width=5),  # Amarillo principal - más grueso
    marker=dict(size=7, color=economist_colors[0], line=dict(color='white', width=2)),
    fill='tozeroy',
    fillcolor='rgba(255, 215, 0, 0.15)',  # Relleno translúcido del amarillo
    name='Desempleo Total',
    hovertemplate='<b>Desempleo Total</b><br>Fecha: %{x}<br>Personas: %{y:,.0f}<extra></extra>'
))

# Añadir línea de tendencia (sin mostrar R²)
import numpy as np
from scipy import stats

x_numeric = np.arange(len(df_temporal))
slope, intercept, r_value, p_value, std_err = stats.linregress(x_numeric, df_temporal["valor"])
trend_line = slope * x_numeric + intercept

fig.add_trace(go.Scatter(
    x=df_temporal["periodo"],
    y=trend_line,
    mode='lines',
    line=dict(color=economist_colors[1], width=4, dash='dash'),  # Azul contrastante
    name='Tendencia',
    hovertemplate='<b>Línea de Tendencia</b><br>Fecha: %{x}<br>Valor: %{y:,.0f}<extra></extra>'
))

# Añadir línea vertical y anotación para inicio COVID-19 en 2020
fig.add_vline(
    x=pd.Timestamp('2020-01-01'),
    line_width=3,
    line_dash="dot",
    line_color=economist_colors[2],  # Gris
)
fig.add_annotation(
    x=pd.Timestamp('2020-01-01'),
    y=max(df_temporal["valor"]),
    text="Inicio COVID-19",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-60,
    font=dict(color=economist_colors[2], size=10, family='Georgia, serif'),  # Tamaño reducido
    bgcolor="white",
    bordercolor=economist_colors[2],
    borderpad=4
)

# Aplicar estilo The Economist
fig = apply_economist_style(fig)

fig.update_layout(
    title="Evolución del Desempleo Total en la Región de Los Ríos",
    xaxis_title="Período",
    yaxis_title="Número de Personas Desocupadas",
    height=500,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        font=dict(size=12)
    )
)

# Formato del eje Y con separadores de miles
fig.update_yaxes(tickformat=',')

fig.show()

print("✅ Gráfico actualizado con fuente de anotación COVID-19 más pequeña")

✅ Gráfico actualizado con fuente de anotación COVID-19 más pequeña


## 📊 Análisis por Tramo Etario

In [60]:
# === ANÁLISIS POR TRAMO ETARIO - SIMPLE Y CLARO ===
print("🔍 LIMPIANDO Y PREPARANDO DATOS...")

# Limpiar y preparar datos
df_edad = df[df["dimension"] == "tramo_edad"].copy()
df_edad["periodo"] = pd.to_datetime(df_edad["periodo"].astype(str).str[:10])

# Eliminar valores atípicos extremos (outliers)
Q1 = df_edad['valor'].quantile(0.25)
Q3 = df_edad['valor'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar datos anómalos
df_edad_limpio = df_edad[(df_edad['valor'] >= limite_inferior) & (df_edad['valor'] <= limite_superior)].copy()

# Etiquetas más simples
etiquetas_simples = {
    "Y15T24": "15-24 años",
    "Y25T34": "25-34 años", 
    "Y35T44": "35-44 años",
    "Y45T54": "45-54 años",
    "Y55T64": "55-64 años",
    "Y_GE65": "65+ años",
    "_T": "Total"
}

df_edad_limpio.loc[:, "grupo"] = df_edad_limpio["categoria"].map(etiquetas_simples)
df_final = df_edad_limpio[df_edad_limpio["grupo"] != "Total"]

print(f"✅ Datos limpios: {len(df_final)} registros")
print(f"📊 Rango de valores: {df_final['valor'].min():,.0f} - {df_final['valor'].max():,.0f}")

# GRÁFICO 1: SOLO LOS 2 GRUPOS MÁS JÓVENES
print("\n📈 GRÁFICO 1: POBLACIÓN JOVEN")

grupos_jovenes = ["15-24 años", "25-34 años"]
fig1 = go.Figure()

for i, grupo in enumerate(grupos_jovenes):
    data = df_final[df_final["grupo"] == grupo]
    
    # Promediar por año para suavizar la línea
    data_anual = data.groupby(data["periodo"].dt.year)["valor"].mean().reset_index()
    data_anual["periodo"] = pd.to_datetime(data_anual["periodo"], format='%Y')
    
    fig1.add_trace(go.Scatter(
        x=data_anual["periodo"],
        y=data_anual["valor"],
        mode='lines+markers',
        line=dict(color=economist_colors[i], width=4),
        marker=dict(size=8),
        name=grupo,
        hovertemplate=f'<b>{grupo}</b><br>Año: %{{x|%Y}}<br>Promedio: %{{y:,.0f}} personas<extra></extra>'
    ))

fig1 = apply_economist_style(fig1)
fig1.update_layout(
    title="Desempleo Juvenil: Población de 15-34 años",
    xaxis_title="Año",
    yaxis_title="Personas Desocupadas (Promedio Anual)",
    height=500,
    showlegend=True
)

fig1.update_yaxes(tickformat=',')
fig1.show()

🔍 LIMPIANDO Y PREPARANDO DATOS...
✅ Datos limpios: 14071 registros
📊 Rango de valores: 9 - 10,415

📈 GRÁFICO 1: POBLACIÓN JOVEN


In [53]:
# GRAFICO 2: POBLACION ADULTA
print("\n[GRAFICO 2] POBLACION ADULTA")

grupos_adultos = ["35-44 años", "45-54 años"]
fig2 = go.Figure()

for i, grupo in enumerate(grupos_adultos):
    data = df_final[df_final["grupo"] == grupo]
    
    # Promediar por año
    data_anual = data.groupby(data["periodo"].dt.year)["valor"].mean().reset_index()
    data_anual["periodo"] = pd.to_datetime(data_anual["periodo"], format='%Y')
    
    fig2.add_trace(go.Scatter(
        x=data_anual["periodo"],
        y=data_anual["valor"],
        mode='lines+markers',
        line=dict(color=economist_colors[i+2], width=4),
        marker=dict(size=8),
        name=grupo,
        hovertemplate=f'<b>{grupo}</b><br>Año: %{{x|%Y}}<br>Promedio: %{{y:,.0f}} personas<extra></extra>'
    ))

fig2 = apply_economist_style(fig2)
fig2.update_layout(
    title="Desempleo Adulto: Poblacion de 35-54 años",
    xaxis_title="Año",
    yaxis_title="Personas Desocupadas (Promedio Anual)",
    height=500,
    showlegend=True
)

fig2.update_yaxes(tickformat=',')
fig2.show()


[GRAFICO 2] POBLACION ADULTA


In [54]:
# GRAFICO 3: POBLACION MAYOR
print("\n[GRAFICO 3] POBLACION MAYOR")

grupos_mayores = ["55-64 años", "65+ años"]
fig3 = go.Figure()

for i, grupo in enumerate(grupos_mayores):
    data = df_final[df_final["grupo"] == grupo]
    
    # Promediar por año
    data_anual = data.groupby(data["periodo"].dt.year)["valor"].mean().reset_index()
    data_anual["periodo"] = pd.to_datetime(data_anual["periodo"], format='%Y')
    
    fig3.add_trace(go.Scatter(
        x=data_anual["periodo"],
        y=data_anual["valor"],
        mode='lines+markers',
        line=dict(color=economist_colors[i+4], width=4),
        marker=dict(size=8),
        name=grupo,
        hovertemplate=f'<b>{grupo}</b><br>Año: %{{x|%Y}}<br>Promedio: %{{y:,.0f}} personas<extra></extra>'
    ))

fig3 = apply_economist_style(fig3)
fig3.update_layout(
    title="Desempleo Mayor: Poblacion de 55+ años",
    xaxis_title="Año",
    yaxis_title="Personas Desocupadas (Promedio Anual)",
    height=500,
    showlegend=True
)

fig3.update_yaxes(tickformat=',')
fig3.show()

# RESUMEN SIMPLE
print(f"\n[RESUMEN] POR GRUPOS:")
print("-" * 40)

for grupo in sorted(df_final["grupo"].unique()):
    promedio = df_final[df_final["grupo"] == grupo]["valor"].mean()
    print(f"• {grupo:12}: {promedio:7,.0f} personas promedio")

print("\n✅ Analisis por edades completado")
print("📌 Datos suavizados por promedios anuales para mayor claridad")


[GRAFICO 3] POBLACION MAYOR



[RESUMEN] POR GRUPOS:
----------------------------------------
• 15-24 años  :   4,140 personas promedio
• 25-34 años  :   4,535 personas promedio
• 35-44 años  :   3,591 personas promedio
• 45-54 años  :   2,768 personas promedio
• 55-64 años  :   1,997 personas promedio
• 65+ años    :     882 personas promedio

✅ Analisis por edades completado
📌 Datos suavizados por promedios anuales para mayor claridad


In [ ]:
# COMPARACIÓN GENERAL: TODOS LOS GRUPOS (ESTILO THE ECONOMIST, MÁS CLARO)
print("\n[GRAFICO GENERAL] COMPARACION DE TODOS LOS GRUPOS - ESTILO MEJORADO")

fig_general = go.Figure()

# Orden y estilos accesibles (mismo color por familia, distinguiendo con línea y marcador)
orden_grupos = [
    "15-24 años", "25-34 años", "35-44 años",
    "45-54 años", "55-64 años", "65+ años"
]

estilos = {
    "15-24 años": {"color": economist_colors[0], "dash": "solid", "marker": "circle"},
    "25-34 años": {"color": economist_colors[1], "dash": "solid", "marker": "square"},
    "35-44 años": {"color": economist_colors[2], "dash": "solid", "marker": "diamond"},
    "45-54 años": {"color": economist_colors[0], "dash": "dash",  "marker": "triangle-up"},
    "55-64 años": {"color": economist_colors[1], "dash": "dash",  "marker": "cross"},
    "65+ años":   {"color": economist_colors[2], "dash": "dash",  "marker": "x"},
}

# Si el dataset no tuviera alguno, usamos los presentes
presentes = sorted(df_final["grupo"].unique())
grupos = [g for g in orden_grupos if g in presentes]

for grupo in grupos:
    data = df_final[df_final["grupo"] == grupo]
    # Promedio anual para claridad
    data_anual = data.groupby(data["periodo"].dt.year)["valor"].mean().reset_index()
    data_anual["periodo"] = pd.to_datetime(data_anual["periodo"], format='%Y')

    estilo = estilos.get(grupo, {"color": economist_colors[2], "dash": "solid", "marker": "circle"})

    fig_general.add_trace(
        go.Scatter(
            x=data_anual["periodo"],
            y=data_anual["valor"],
            mode="lines+markers",
            line=dict(color=estilo["color"], width=3.5, dash=estilo["dash"]),
            marker=dict(size=7, symbol=estilo["marker"], line=dict(color="white", width=1.5)),
            name=grupo,
            hovertemplate=f"<b>{grupo}</b><br>Año: %{{x|%Y}}<br>Promedio: %{{y:,.0f}} personas<extra></extra>",
        )
    )

fig_general = apply_economist_style(fig_general)
fig_general.update_layout(
    title="Comparación general: Desempleo por grupos etarios (promedio anual)",
    xaxis_title="Año",
    yaxis_title="Personas desocupadas (promedio anual)",
    height=600,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        title="Grupos etarios",
    ),
)

fig_general.update_yaxes(tickformat=',')
fig_general.show()


[GRAFICO GENERAL] COMPARACION DE TODOS LOS GRUPOS


In [55]:
# Usar datos limpios de la sección anterior
df_resumen_limpio = df_final.groupby("periodo")["valor"].sum().reset_index()

# Crear gráfico con área rellena y colores daltónicos
fig = go.Figure()

# Añadir área principal con color daltónico
fig.add_trace(go.Scatter(
    x=df_resumen_limpio["periodo"],
    y=df_resumen_limpio["valor"],
    mode='lines',
    line=dict(color=economist_colors[0], width=4),  # Rojo daltónico
    fill='tozeroy',
    fillcolor=f'rgba(232, 68, 58, 0.15)',  # Relleno translúcido
    name='Desempleo Total',
    hovertemplate='<b>Fecha:</b> %{x}<br><b>Total Personas:</b> %{y:,.0f}<extra></extra>'
))

# Añadir línea de tendencia con color daltónico
import numpy as np
from scipy import stats

x_numeric = np.arange(len(df_resumen_limpio))
slope, intercept, r_value, p_value, std_err = stats.linregress(x_numeric, df_resumen_limpio["valor"])
trend_line = slope * x_numeric + intercept

fig.add_trace(go.Scatter(
    x=df_resumen_limpio["periodo"],
    y=trend_line,
    mode='lines',
    line=dict(color=economist_colors[8], width=3, dash='dash'),  # Gris daltónico
    name=f'Tendencia (R²={r_value**2:.3f})',
    hovertemplate='<b>Línea de Tendencia</b><br>Fecha: %{x}<br>Valor: %{y:,.0f}<extra></extra>'
))

# Aplicar estilo The Economist
fig = apply_economist_style(fig)

fig.update_layout(
    title="Resumen: Desempleo Total por Tramos Etarios con Tendencia",
    xaxis_title="Período",
    yaxis_title="Número Total de Personas Desocupadas",
    height=500,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)

fig.update_yaxes(tickformat=',')
fig.show()

## 👥 Análisis por Sexo

Comparación del desempleo entre hombres y mujeres en la región.

In [ ]:
# Preparar datos por sexo (incluye Total)
df_sexo = df[df["dimension"] == "sexo"].copy()
df_sexo["periodo"] = pd.to_datetime(df_sexo["periodo"].astype(str).str[:10])

# Mapear códigos de sexo
etiquetas_sexo = {
    "F": "Mujeres",
    "M": "Hombres",
    "_T": "Total"
}

df_sexo["sexo_label"] = df_sexo["categoria"].map(etiquetas_sexo)
# Incluir Hombres, Mujeres y Total
orden_series = ["Hombres", "Mujeres", "Total"]
df_sexo_filtrado = df_sexo[df_sexo["sexo_label"].isin(orden_series)]

# Crear gráfico comparativo con estilo más claro (líneas más delgadas)
fig = go.Figure()

colores_sexo_daltonico = {
    "Hombres": economist_colors[1],   # Azul
    "Mujeres": economist_colors[0],   # Amarillo
    "Total":   economist_colors[2]    # Gris
}

for sexo in orden_series:
    data_sexo = df_sexo_filtrado[df_sexo_filtrado["sexo_label"] == sexo]
    if data_sexo.empty:
        continue

    if sexo == "Total":
        # Total: línea discontinua, sin marcadores, un poco tenue
        fig.add_trace(go.Scatter(
            x=data_sexo["periodo"],
            y=data_sexo["valor"],
            mode='lines',
            line=dict(color=colores_sexo_daltonico[sexo], width=2.2, dash='dash'),
            opacity=0.9,
            name=sexo,
            hovertemplate=f'<b>{sexo}</b><br>Fecha: %{{x}}<br>Personas: %{{y:,.0f}}<extra></extra>'
        ))
    else:
        # Hombres/Mujeres: líneas + marcadores, más delgadas y marcadores pequeños
        fig.add_trace(go.Scatter(
            x=data_sexo["periodo"],
            y=data_sexo["valor"],
            mode='lines+markers',
            line=dict(color=colores_sexo_daltonico[sexo], width=2.4),
            marker=dict(size=5, color=colores_sexo_daltonico[sexo], line=dict(color='white', width=1)),
            name=sexo,
            hovertemplate=f'<b>{sexo}</b><br>Fecha: %{{x}}<br>Personas: %{{y:,.0f}}<extra></extra>'
        ))

# Aplicar estilo The Economist
fig = apply_economist_style(fig)

fig.update_layout(
    title="Comparación del Desempleo por Sexo - Región de Los Ríos",
    xaxis_title="Período",
    yaxis_title="Número de Personas Desocupadas",
    height=500,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        font=dict(size=12)
    )
)

fig.update_yaxes(tickformat=',')
fig.show()

## 📈 Resumen Estadístico y Tendencias Clave

Principales hallazgos del análisis exploratorio.

## 🎨 Verificación de Paleta de Colores Accesible

Demostración de la paleta de colores utilizada, diseñada para facilitar la visualización y comprensión de los datos.

In [11]:
# Calcular estadísticas resumen
print("📊 RESUMEN ESTADÍSTICO - DESEMPLEO REGIÓN DE LOS RÍOS")
print("=" * 60)

# Usar datos ya procesados de las celdas anteriores
# 1. Estadísticas generales del período
print(f"\n🔍 DATOS GENERALES:")
print(f"• Período analizado: {df_temporal['periodo'].min().strftime('%Y-%m')} a {df_temporal['periodo'].max().strftime('%Y-%m')}")
print(f"• Promedio de desocupados: {df_temporal['valor'].mean():,.0f} personas")
print(f"• Máximo histórico: {df_temporal['valor'].max():,.0f} personas")
print(f"• Mínimo histórico: {df_temporal['valor'].min():,.0f} personas")

# 2. Análisis por sexo (usando datos procesados)
print(f"\n👥 ANÁLISIS POR SEXO:")
sexo_resumen = df_sexo_filtrado.groupby("sexo_label")["valor"].mean()
for sexo, promedio in sexo_resumen.items():
    print(f"• {sexo}: {promedio:,.0f} personas promedio")

# 3. Análisis por tramo etario
print(f"\n📊 GRUPOS ETARIOS MÁS AFECTADOS:")
promedio_por_edad = df_edad_filtrado.groupby("categoria_label")["valor"].mean().sort_values(ascending=False)
for i, (grupo, promedio) in enumerate(promedio_por_edad.head(3).items()):
    print(f"{i+1}. {grupo}: {promedio:,.0f} personas promedio")

# 4. Tendencia general usando datos ya procesados
print(f"\n🎯 TENDENCIA GENERAL:")
primer_valor = df_temporal['valor'].iloc[0]
ultimo_valor = df_temporal['valor'].iloc[-1]
cambio_porcentual = ((ultimo_valor - primer_valor) / primer_valor) * 100

if cambio_porcentual > 0:
    print(f"📈 CRECIENTE: +{cambio_porcentual:.1f}% desde el inicio del período")
else:
    print(f"📉 DECRECIENTE: {cambio_porcentual:.1f}% desde el inicio del período")

# 5. Comparación por sexo
diff_sexo = abs(sexo_resumen["Mujeres"] - sexo_resumen["Hombres"])
sexo_mayor = "Mujeres" if sexo_resumen["Mujeres"] > sexo_resumen["Hombres"] else "Hombres"
print(f"• Brecha de género: {diff_sexo:,.0f} personas más en {sexo_mayor}")

print("\n" + "=" * 60)
print("✅ Análisis completado exitosamente")

📊 RESUMEN ESTADÍSTICO - DESEMPLEO REGIÓN DE LOS RÍOS

🔍 DATOS GENERALES:
• Período analizado: 2010-01 a 2025-02
• Promedio de desocupados: 279,049 personas
• Máximo histórico: 520,387 personas
• Mínimo histórico: 200,728 personas

👥 ANÁLISIS POR SEXO:
• Hombres: 6,101 personas promedio
• Mujeres: 6,281 personas promedio

📊 GRUPOS ETARIOS MÁS AFECTADOS:
1. 25-34 años: 5,253 personas promedio
2. 15-24 años: 4,220 personas promedio
3. 35-44 años: 3,779 personas promedio

🎯 TENDENCIA GENERAL:
📈 CRECIENTE: +43.4% desde el inicio del período
• Brecha de género: 180 personas más en Mujeres

✅ Análisis completado exitosamente


## 👥 Comparación del Desempleo por Sexo

Este gráfico muestra la evolución del desempleo en la Región de Los Ríos diferenciando entre hombres y mujeres. Se utilizan colores amarillo y azul para distinguir ambos grupos, siguiendo la paleta accesible definida para todo el análisis.

- **Hombres:** Línea azul
- **Mujeres:** Línea amarilla

La visualización permite identificar tendencias y posibles brechas de género en el desempleo a lo largo del tiempo. El estilo The Economist asegura claridad y profesionalismo en la presentación.